In [ ]:
# Start writing cod# load init moudle file

$data_import = true
load "/work/pannote/panbot_note.rb"


true

In [ ]:
code = <<~'TASKCODE'

# save to main-db
require 'active_record'
require 'faraday'
require 'ethereum.rb'
require 'faraday'
require 'parallel'
require 'resolv-replace'

connect_str = __PARAMS__CONNECT_STR__
ActiveRecord::Base.establish_connection(connect_str)


## load data model from github
models = ["application_record","address","cache","block","epoch","epoch_detail","event","transfer","tx","log"]
models.each do |model| 
  url = "https://raw.githubusercontent.com/adam429/panscan/main/panscan/app/models/#{model}.rb"
  body = Faraday.get(url).body
  f = open("load.rb","w")
  f.write(body)
  f.close
  load './load.rb'
end

## 数据准备


def calc_bot_stats(address)
  ar_addr = Address.find_by_addr(address)
  return if ar_addr.bet_cnt!=nil
  
  txes = Tx.where('"from" = ?',address).where("method_name=? or method_name=?",'betBull','betBear').order(:block_number)

  tx_map = {}
  Event.where(tx_hash:txes.map {|x| x.tx_hash}).each {|event|
      amount = JSON.parse(event.params)["amount"]
      tx_map[event.tx_hash] = [
         event.name,
         amount,
         event.block_number
     ]
  }        

  epoch_map = {}

  tx_block_map = txes.map {|x| [x.tx_hash,x.block_number]}.to_h
  block_epoch_map = Block.where(block_number:txes.map {|x| x.block_number}).map {|x| [x.block_number,x.epoch]}.to_h
  epoch_ar_map = Epoch.where(epoch:block_epoch_map.to_a.map{|x| x[1]}).map {|x| [x.epoch,x]}.to_h

  tx_block_map.each { |k,v| 
      epoch_map[k] = epoch_ar_map[block_epoch_map[v]]
  }

    avg_last_block_order = []
    wrong_bet_list =[]
    win_bet_list = []
    avg_amount = []
    invest_cnt = 0
    invest_amt = 0
    return_amt = 0

    Parallel.map(txes,in_threads: 10) do |tx|
      round_ret = 0
      x = tx_map[tx.tx_hash]
      avg_amount.push(x[1]) if x!=nil

      epoch = epoch_map[tx.tx_hash]
      next if epoch==nil

      last_block_order = epoch.get_last_block_order(tx.block_number)

      next if last_block_order==nil
      avg_last_block_order.push(last_block_order) if tx.tx_status 
      try_count = 3

begin
      wrong_bet = epoch.get_wrong_bet(tx.method_name,tx.block_number);
      try_count = try_count-1
rescue NoMethodError=>e
  sleep(1)
  retry if try_count>0
  puts tx.inspect
  raise NoMethodError
end

      wrong_bet_list.push wrong_bet
      win_bet = tx.method_name[-4,4].downcase == epoch.bet_result 
      win_bet_list.push win_bet

      if tx.tx_status then
        invest_cnt = invest_cnt +1
        bet_amt = (tx_map[tx.tx_hash] or [0,0,0])[1]
        invest_amt = invest_amt + bet_amt

        round_ret =  - bet_amt
        if tx.method_name=="betBear" and epoch.bet_result=="bear" then
          round_ret = round_ret+ bet_amt * epoch.bear_payout
        end
        if tx.method_name=="betBull" and epoch.bet_result=="bull" then
          round_ret = round_ret+bet_amt * epoch.bull_payout
        end
        return_amt = return_amt + round_ret

      end
  end
  ar_addr.bet_epoch_cnt = txes.map {|x| x.block ? x.block.epoch : 0}.uniq.size
  ar_addr.invest_cnt = txes.where(tx_status:true).size
  ar_addr.bet_cnt = txes.size
  ar_addr.bet_bull_cnt = txes.where(method_name:"betBull").size
  ar_addr.bet_bear_cnt = txes.where(method_name:"betBear").size
  ar_addr.avg_bet_amt = avg_amount.sum / avg_amount.size.to_f
  ar_addr.bet_amt = JSON.dump(avg_amount)
  ar_addr.avg_last_block_order = avg_last_block_order.sum / avg_last_block_order.size.to_f
  ar_addr.right_bet_ratio = wrong_bet_list.filter{|x| x==false}.size / wrong_bet_list.size.to_f
  ar_addr.win_bet_ratio = win_bet_list.filter{|x| x==true}.size / win_bet_list.size.to_f
  ar_addr.invest_amt = invest_amt
  ar_addr.return_amt = return_amt
  ar_addr.save
end

puts "#{Time.now} start"
addr_list = __PARAMS__ADDR_LIST__
addr_list.map do |address|
  calc_bot_stats(address)
end

puts "#{Time.now} end"

TASKCODE



"\n# save to main-db\nrequire 'active_record'\nrequire 'faraday'\nrequire 'ethereum.rb'\nrequire 'faraday'\nrequire 'parallel'\nrequire 'resolv-replace'\n\nconnect_str = __PARAMS__CONNECT_STR__\nActiveRecord::Base.establish_connection(connect_str)\n\n\n## load data model from github\nmodels = [\"application_record\",\"address\",\"cache\",\"block\",\"epoch\",\"epoch_detail\",\"event\",\"transfer\",\"tx\",\"log\"]\nmodels.each do |model| \n  url = \"https://raw.githubusercontent.com/adam429/panscan/main/panscan/app/models/\#{model}.rb\"\n  body = Faraday.get(url).body\n  f = open(\"load.rb\",\"w\")\n  f.write(body)\n  f.close\n  load './load.rb'\nend\n\n## 数据准备\n\n\ndef calc_bot_stats(address)\n  ar_addr = Address.find_by_addr(address)\n  return if ar_addr.bet_cnt!=nil\n  \n  txes = Tx.where('\"from\" = ?',address).where(\"method_name=? or method_name=?\",'betBull','betBear').order(:block_number)\n\n  tx_map = {}\n  Event.where(tx_hash:txes.map {|x| x.tx_hash}).each {|event|\n      amoun

In [ ]:
addr_list = Address.where(is_panbot:true).order(:id).select(:addr,:id,:bet_cnt).map {|x| [x.id,x.addr,x.bet_cnt] }
addr_list = addr_list.filter {|x| x[2]==nil}
addr_list = addr_list.map.with_index.to_a.map {|x| [x[0][0],x[0][1],x[0][2],x[1]] }

[]

In [ ]:
begin_id = addr_list.first[3]
end_id = addr_list.last[3]
step = 1000

loop do
  task_end_id = [(begin_id+step)/1000*1000,begin_id+step,end_id].min
  task_name = "calc_bot_stats #{begin_id} - #{task_end_id}"

  puts task_name
  task_code = code.gsub(/__PARAMS__CONNECT_STR__/,'"'+ENV["DB_CONNECT_STR"]+'"')
                  .gsub(/__PARAMS__ADDR_LIST__/,"[#{ addr_list[begin_id,task_end_id-begin_id+1].map{|x| '\''+x[1]+'\''}.join(',') } ]" )
  Task.create_task(task_name, task_code)

  begin_id = [(begin_id+step)/1000*1000,begin_id+step].min
  break if begin_id > end_id
end


calc_bot_stats 0 - 100
calc_bot_stats 100 - 200
calc_bot_stats 200 - 300
calc_bot_stats 300 - 400
calc_bot_stats 400 - 500
calc_bot_stats 500 - 600
calc_bot_stats 600 - 700
calc_bot_stats 700 - 800
calc_bot_stats 800 - 900
calc_bot_stats 900 - 1000
calc_bot_stats 1000 - 1100
calc_bot_stats 1100 - 1200
calc_bot_stats 1200 - 1300
calc_bot_stats 1300 - 1400
calc_bot_stats 1400 - 1500
calc_bot_stats 1500 - 1600
calc_bot_stats 1600 - 1700
calc_bot_stats 1700 - 1800
calc_bot_stats 1800 - 1900
calc_bot_stats 1900 - 2000
calc_bot_stats 2000 - 2100
calc_bot_stats 2100 - 2200
calc_bot_stats 2200 - 2300
calc_bot_stats 2300 - 2400
calc_bot_stats 2400 - 2500
calc_bot_stats 2500 - 2600
calc_bot_stats 2600 - 2700
calc_bot_stats 2700 - 2800
calc_bot_stats 2800 - 2900
calc_bot_stats 2900 - 3000
calc_bot_stats 3000 - 3100
calc_bot_stats 3100 - 3200
calc_bot_stats 3200 - 3300
calc_bot_stats 3300 - 3400
calc_bot_stats 3400 - 3500
calc_bot_stats 3500 - 3600
calc_bot_stats 3600 - 3700
calc_bot_stats 3700 - 

In [ ]:
Address.find_by_addr("0x81789ce3db8e4eb0cd084e055c90da4f37843c55")

#<Address id: 916225, addr: "0x81789ce3db8e4eb0cd084e055c90da4f37843c55", is_contract: false, contract_abi: nil, is_panbot: true, bet_cnt_daily: nil, bet_epoch_cnt: 9, invest_cnt: 9, bet_cnt: 9, bet_bull_cnt: 6, bet_bear_cnt: 3, avg_bet_amt: 0.07, bet_amt: "[0.05,0.05,0.05,0.05,0.1,0.2,0.05,0.03,0.05]", right_bet_ratio: 0.2222222222222222, win_bet_ratio: 0.4444444444444444, invest_amt: 0.6300000000000001, return_amt: -0.26074671789579507, created_at: "2022-02-15 00:14:27.716693000 +0000", updated_at: "2022-02-15 03:24:58.066399000 +0000", tag: nil, avg_last_block_order: 33.333333333333336>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>